## Assignment 2
### Problem 6.
For this problem, you will use each of the three models you’ve constructed in problems 2–4 to
evaluate the probability of a toy corpus of sentences (containing just the first two sentences of the
Brown corpus) found at: toy_corpus.txt

The sentences are contained in this corpus in the same format as the other corpora you’ve been
using so far: one sentence per line and words separated by a space. As before, you’ll need to remove
the end-of-line character.

In [107]:
# Problem 2,  wouter's version
##
import nltk
import numpy as np
from nltk.corpus import brown
from collections import Counter
from nltk import pos_tag

vocab = open("brown_vocab_100.txt")

#load the indices dictionary
word_index_dict = {}
for i, line in enumerate(vocab):
    #import part 1 code to build dictionary
    word_index_dict[line.rstrip()] = i

f = open("brown_100.txt")

counts = np.zeros(len(word_index_dict)) #initialize counts to a zero vector

#iterate through file and update counts
for sent in f:
    sent_list = sent.lower().split()

    for word in sent_list:
        ind = word_index_dict[word]
        counts[ind] += 1

f.close()

#normalize and writeout counts. 

#print(counts) edited for space
probs = counts / np.sum(counts)
print(probs[0])
print(probs[-1])

0.0004051863857374392
0.003646677471636953


#### *Beginning of Problem 6*
First, you’ll edit problem2.py, and add code at the bottom of the script to iterate through each
sentence in the toy corpus and calculate the joint probability of all the words in the sentence under
the unigram model. Then write the probability of each sentence to a file unigram_eval.txt,
formatted to have one probability for each line of the output file.

To do this, you’ll be writing a loop within a loop very similarly to how you iterated through the
training corpus to estimate the model parameters. But instead of incrementing counts after each
word, you’ll be updating the joint probability of the sentence (multiplying the probabilities of each
word together). One easy way to do this is to initialize sentprob = 1 prior to looping through
the words in the sentence, and then just update sentprob *= wordprob with the probability
of each word. At the end of the loop, sentprob will contain the total joint probability of the whole
sentence. To verify that this worked correctly, note that the joint probability of the second sentence
under this model should be 4.84008387782e-99

Next, you’ll transform this joint probability into a perplexity (of each sentence), and write that to the
file instead. To calculate the perplexity, first calculate the length of the sentence in words (be sure to
include the end-of-sentence word) and store that in a variable sent_len, and then you can
calculate perplexity = 1/(pow(sentprob, 1.0/sent_len)), which reproduces the
definition of perplexity we discussed in class.

Now, write the perplexity of each sentence to the output file instead of the joint probabilities. To
verify that you’ve done this correctly, note that the perplexity of the second sentence with this
model should be about 153.

### MLE unigram model (wouter version) (problem 2):


In [109]:
#open and rea dthe toy_corpus.txt
toy_corpus_file = open("toy_corpus.txt", "r")
toy_corpus_text = toy_corpus_file.read()
toy_corpus_file.close()
sentences = toy_corpus_text.split('\n')

#open to write the probabilities into the new txt file
output_file = open("unigram_eval.txt", "w")

#iterate thru sentences and calculate joint prob of each one
for sentence in sentences:
    words = sentence.split()
    #initialize sentence prob
    sentprob = 1
    #computes the joint prob per word in sentence
    for word in words:
        #finds index of word
        index = word_index_dict.get(word.lower(), None)
        if index is not None:
            #finds prob of word
            wordprob = probs[index]
            #updated sentence prob by * with word prob
            sentprob *= wordprob
    #To calculate the perplexity, first calculate the length of the sentence in words (be sure to
    #include the end-of-sentence word)  
    #not sure why the perplexity is off without counting the <s>
    sent_len =len(words) - words.count("<s>")  + 1
    #calculate perplexity
    perplexity = 1/(pow(sentprob, 1.0/sent_len))    
    
    #added the perplexity check to unigram_eval.txt (instead of joint prob, to check joint, have it write
    #output_file.write(str(sentprob) + '\n') instead
    output_file.write(str(perplexity) + '\n')

#close file
output_file.close()

#verify jopint pro b is 4.84008387782
print('Joint probability of the 2nd sentence: ', sentprob)

print('sent_len: ', sent_len)
print('sentprob: ', sentprob)
#cheeck perplexity of 2nd sentence is 153
print('perplexity of the 2nd sentence: ', perplexity)

Joint probability of the 2nd sentence:  4.840083877820268e-99
sent_len:  45
sentprob:  4.840083877820268e-99
perplexity of the 2nd sentence:  153.03157461392973


### MLE bigram model: (wouter version) (problem 3)

In [133]:
import numpy as np
from sklearn.preprocessing import normalize
import codecs
# Read the toy corpus
toy_corpus_file = open("toy_corpus.txt", "r")
toy_corpus_text = toy_corpus_file.read()
toy_corpus_file.close()
sentences = toy_corpus_text.split('\n')

vocab = codecs.open("brown_vocab_100.txt")

#load the indices dictionary
word_index_dict = {}
for i, line in enumerate(vocab):
    #import part 1 code to build dictionary
    word_index_dict[line.rstrip()] = i

f = codecs.open("brown_100.txt")

counts = np.zeros((len(word_index_dict), len(word_index_dict))) #initialize numpy 0s matrix


#iterate through file and update counts
prev_word = "<s>"

for sent in f:
    sent_list = sent.lower().split()
    for word in sent_list:
        next_word = word
        ind_y = word_index_dict[next_word]
        ind_x = word_index_dict[prev_word]
        counts[ind_x, ind_y] += 1
        prev_word = next_word      

# normalize counts
probs = normalize(counts, norm='l1', axis=1)

#writeout bigram probabilities
the_ind = word_index_dict["the"]
all_ind = word_index_dict["all"]
jury_ind = word_index_dict["jury"]
campaign_ind = word_index_dict["campaign"]
calls_ind = word_index_dict["calls"]
anonymous_ind = word_index_dict["anonymous"]

fourprobs = [probs[all_ind, the_ind],
                probs[the_ind, jury_ind],
                probs[the_ind, campaign_ind],
                probs[anonymous_ind, calls_ind]]
print(fourprobs)

with open("bigram_probs.txt", 'w') as file:
    for p in fourprobs:
        file.write(f"{p}\n")
f.close()

#start and open the perplexity file
output_file = open("bigram_eval.txt", "w")

#perplexities calculations for toy corpus
#iterates through sentences
for sent in sentences:
    words = sent.lower().split()
    sentprob = 1
    prev_word = "<s>"
    for word in words:
        next_word = word
        ind_y = word_index_dict[next_word]
        ind_x = word_index_dict[prev_word]
        if ind_x is not None and ind_y is not None:
            wordprob = probs[ind_x, ind_y]
            sentprob *= wordprob
        prev_word = next_word
    sent_len =len(words) + 1
    #calculate perplexity
    perplexity = 1/(pow(sentprob, (1.0/sent_len)))  
    output_file.write(str(perplexity) + '\n')

#close file
output_file.close()

print('sent_len: ', sent_len)
print('sentprob: ', sentprob)
#cheeck perplexity of 2nd sentence is about 7.57 for the MLE bigram
print('perplexity of the 2nd sentence for MLE bigram is: ', perplexity)

[1.0, 0.08333333333333333, 0.00641025641025641, 0.3333333333333333]
sent_len:  46
sentprob:  2.0462234274543635e-41
perplexity of the 2nd sentence for MLE bigram is:  7.6655707941796365


### MLE smoother model (wouter version) (problem 4):

In [139]:
import numpy as np
from sklearn.preprocessing import normalize
import codecs
# Read the toy corpus
toy_corpus_file = open("toy_corpus.txt", "r")
toy_corpus_text = toy_corpus_file.read()
toy_corpus_file.close()
#sentences = toy_corpus_text.split('\n')

vocab = codecs.open("brown_vocab_100.txt")

#load the indices dictionary
word_index_dict = {}
for i, line in enumerate(vocab):
    #import part 1 code to build dictionary
    word_index_dict[line.rstrip()] = i

f = codecs.open("brown_100.txt")

counts = np.zeros((len(word_index_dict), len(word_index_dict))) #initialize numpy 0s matrix

#iterate through file and update counts
prev_word = "<s>"

for sent in f:
    sent_list = sent.lower().split()
    for word in sent_list:
        next_word = word
        ind_y = word_index_dict[next_word]
        ind_x = word_index_dict[prev_word]
        counts[ind_x, ind_y] += 1
        prev_word = next_word

counts += 0.1

# normalize counts
probs = normalize(counts, norm='l1', axis=1)

#writeout bigram probabilities
the_ind = word_index_dict["the"]
all_ind = word_index_dict["all"]
jury_ind = word_index_dict["jury"]
campaign_ind = word_index_dict["campaign"]
calls_ind = word_index_dict["calls"]
anonymous_ind = word_index_dict["anonymous"]

fourprobs = [probs[all_ind, the_ind],
                probs[the_ind, jury_ind],
                probs[the_ind, campaign_ind],
                probs[anonymous_ind, calls_ind]]

print(fourprobs)

with open("smooth_probs.txt", 'w') as file:
    for p in fourprobs:
        file.write(f"{p}\n")

f.close()


#start and open the perplexity file
output_file = open("smoothed_eval.txt", "w")

# Reopen toy corpus file to read from the beginning
toy_corpus_file = open("toy_corpus.txt", "r")

#perplexities calculations for toy corpus
#iterates through sentences
for sent in toy_corpus_file:
    words = sent.lower().split()
    sentprob = 1
    prev_word = "<s>"
    for word in words:
        next_word = word
        ind_y = word_index_dict[next_word]
        ind_x = word_index_dict[prev_word]
        if ind_x is not None and ind_y is not None:
            wordprob = probs[ind_x, ind_y]
            sentprob *= wordprob
        prev_word = next_word
    # added back in the  - words.count("<s>") as its closer in terms of perplexity, but cant figure out difference between the two problems why..
    #sent_len is now 45, compared to 46 in the MLE bigram... and 45 in the unigram. 
    sent_len =len(words) - words.count("<s>") + 1
    #calculate perplexity
    perplexity = 1/(pow(sentprob, (1.0/sent_len)))  
    output_file.write(str(perplexity) + '\n')

#close file
output_file.close()
toy_corpus_file.close()

print('sent_len: ', sent_len)
print('sentprob: ', sentprob)
#cheeck perplexity of 2nd sentence is about 54.28 for the alpha-smoothed model
print('perplexity of the 2nd sentence for alpha-smoothed model is: ', perplexity)

[0.01336573511543135, 0.05520438263801095, 0.004635482511588706, 0.013048635824436538]
sent_len:  45
sentprob:  2.949110312537236e-79
perplexity of the 2nd sentence for alpha-smoothed model is:  55.60552908217579
